In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
import cv2
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import Xception


In [ ]:
!kaggle datasets download -d fanconic/skin-cancer-malignant-vs-benign

Dataset URL: https://www.kaggle.com/datasets/fanconic/skin-cancer-malignant-vs-benign
License(s): unknown
 98% 319M/325M [00:02<00:00, 160MB/s]
100% 325M/325M [00:02<00:00, 131MB/s]


In [ ]:
!mkdir items
!unzip skin-cancer-malignant-vs-benign -d /content/items

Streaming output truncated to the last 5000 lines.
  inflating: /content/items/data/train/benign/420.jpg  
  inflating: /content/items/data/train/benign/421.jpg  
  inflating: /content/items/data/train/benign/422.jpg  
  inflating: /content/items/data/train/benign/423.jpg  
  inflating: /content/items/data/train/benign/424.jpg  
  inflating: /content/items/data/train/benign/425.jpg  
  inflating: /content/items/data/train/benign/426.jpg  
  inflating: /content/items/data/train/benign/429.jpg  
  inflating: /content/items/data/train/benign/43.jpg  
  inflating: /content/items/data/train/benign/430.jpg  
  inflating: /content/items/data/train/benign/431.jpg  
  inflating: /content/items/data/train/benign/432.jpg  
  inflating: /content/items/data/train/benign/434.jpg  
  inflating: /content/items/data/train/benign/435.jpg  
  inflating: /content/items/data/train/benign/438.jpg  
  inflating: /content/items/data/train/benign/439.jpg  
  inflating: /content/items/data/train/benign/440.jpg 

In [ ]:

ben = os.listdir('/content/items/train/benign')
mal = os.listdir('/content/items/train/malignant')

# Let benign be 0 and malignant be 1
train = []
train_y = []


In [ ]:

# loading the dataset
for i in ben:
    x = '/content/items/train/benign/' + i
    img = cv2.imread(x)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(200,200))
    img = img/255 # normalising
    train.append(img.flatten())
    train_y.append(0)

for i in mal:
    x = '/content/items/train/malignant/' + i
    img = cv2.imread(x)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(200,200))
    img = img/255 # normalising
    train.append(img.flatten())
    train_y.append(1)



In [ ]:
train = np.array(train)


In [ ]:

# Splitting the dataset
train,val,train_y,val_y = train_test_split(train,train_y,test_size=0.2,random_state=44)
train = train.reshape(train.shape[0],200,200,3)
val = val.reshape(val.shape[0],200,200,3)
encoder = LabelEncoder()
encoder = encoder.fit(train_y)
train_y = encoder.transform(train_y)
encoder = encoder.fit(val_y)
val_y = encoder.transform(val_y)
print(str('training rows ' + str(len(train))))
print(str('validation rows ' + str(len(val))))

# neural network model creation
model = Sequential()
base = Xception(include_top=False,weights="imagenet",input_shape=(200,200,3),pooling='avg')
for lay in base.layers: lay.trainable = True
model.add(base)
model.add(Dropout(0.8))
model.add(Flatten())
model.add(Dense(64,'relu'))
model.add(Dropout(0.6))
model.add(Dense(1,'sigmoid'))
model.compile(Adam(0.0001),'binary_crossentropy',['accuracy'])
model.summary()


training rows 2109
validation rows 528
83683744/83683744 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                  

In [ ]:

# defining callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss',patience=2,verbose=2,factor=0.3,min_lr=0.000001)
early_stop = EarlyStopping(patience=4,restore_best_weights=True)

# training the model
model.fit(train,train_y,epochs=25,batch_size=10,validation_data=(val,val_y),verbose=2,callbacks=[early_stop,reduce_lr])

# testing the trained model
ben = os.listdir('/content/items/test/benign')
mal = os.listdir('/content/items/test/malignant')
test = []
test_y = []


Epoch 1/25
211/211 - 67s - loss: 0.5983 - accuracy: 0.6799 - val_loss: 0.3553 - val_accuracy: 0.8428 - lr: 1.0000e-04 - 67s/epoch - 317ms/step
Epoch 2/25
211/211 - 30s - loss: 0.3125 - accuracy: 0.8653 - val_loss: 0.2629 - val_accuracy: 0.8845 - lr: 1.0000e-04 - 30s/epoch - 141ms/step
Epoch 3/25
211/211 - 31s - loss: 0.2593 - accuracy: 0.8971 - val_loss: 0.3190 - val_accuracy: 0.8504 - lr: 1.0000e-04 - 31s/epoch - 146ms/step
Epoch 4/25

Epoch 4: ReduceLROnPlateau reducing learning rate to 2.9999999242136255e-05.
211/211 - 31s - loss: 0.1735 - accuracy: 0.9360 - val_loss: 0.3520 - val_accuracy: 0.8485 - lr: 1.0000e-04 - 31s/epoch - 148ms/step
Epoch 5/25
211/211 - 31s - loss: 0.1077 - accuracy: 0.9711 - val_loss: 0.2504 - val_accuracy: 0.8958 - lr: 3.0000e-05 - 31s/epoch - 148ms/step
Epoch 6/25
211/211 - 31s - loss: 0.0715 - accuracy: 0.9772 - val_loss: 0.2848 - val_accuracy: 0.8845 - lr: 3.0000e-05 - 31s/epoch - 147ms/step
Epoch 7/25

Epoch 7: ReduceLROnPlateau reducing learning rate to

In [ ]:
# import matplotlib.pyplot as plt
# import random

# # Combine train, val, and test datasets
# all_data = np.concatenate((train, val, test), axis=0)
# all_labels = np.concatenate((train_y, val_y, test_y), axis=0)

# # Choose a random image from the dataset
# random_index = random.randint(0, len(all_data) - 1)
# random_image = all_data[random_index]
# random_label = all_labels[random_index]

# # Plot the image
# plt.imshow(random_image)
# plt.title('Random Image from Dataset')
# plt.show()




ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 4 dimension(s) and the array at index 2 has 1 dimension(s)

In [ ]:
# # Reshape the image to match the input shape of the model
# random_image = random_image.reshape(1, 200, 200, 3)

# # Use the model to predict whether the image is benign or malignant
# prediction = model.predict(random_image)

# # Print the prediction
# if prediction < 0.5:
#     print("The model predicts this image is benign.")
# else:
#     print("The model predicts this image is malignant.")

# # Print the actual label
# if random_label == 0:
#     print("The actual label of this image is benign.")
# else:
#     print("The actual label of this image is malignant.")

NameError: name 'random_image' is not defined

In [ ]:

for i in ben:
    x = '/content/items/test/benign/' + i
    img = cv2.imread(x)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(200,200))
    img = img/255 # normalising
    test.append(img)
    test_y.append(0)
for i in mal:
    x = '/content/items/test/malignant/' + i
    img = cv2.imread(x)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(200,200))
    img = img/255 # normalising
    test.append(img)
    test_y.append(1)

test = np.array(test)
encoder = LabelEncoder()
encoder = encoder.fit(test_y)
test_y = encoder.transform(test_y)

loss,acc = model.evaluate(test, test_y,verbose=2)

print('Accuracy on test data: '+ str(acc*100))
print('Loss on test data: ' + str(loss))

21/21 - 7s - loss: 0.3037 - accuracy: 0.8727 - 7s/epoch - 352ms/step
Accuracy on test data: 87.27272748947144
Loss on test data: 0.30372154712677
